For more details on this tutorial's setup and process, see `00_Introduction.ipynb`.

# Stage 3: Deploy the Model in Wallaroo
 
In this stage, we upload the trained model and the processing steps to Wallaroo, then set up and deploy the inference pipeline. 

Once deployed we can feed the newest batch of data to the pipeline, do the inferences and write the results to a results table.

For clarity in this demo, we have split the training/upload task into two notebooks:

* `02_automated_training_process.ipynb`: Train and pickle ML model.
* `03_deploy_model.ipynb`: Upload the model to Wallaroo and deploy into a pipeline.

Assuming no changes are made to the structure of the model, these two notebooks, or a script based on them, can then be scheduled to run on a regular basis, to refresh the model with more recent training data and update the inference pipeline.

This notebook is expected to run within the Wallaroo instance's Jupyter Hub service to provide access to all required Wallaroo libraries and functionality.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `postprocess.py`: Formats the data after inference by the model is complete.
  * `preprocess.py`: Formats the incoming data for the model.
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
  * `wallaroo_client.py`: Additional methods used with the Wallaroo instance to create workspaces, etc.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.

## Steps

The process of uploading the model to Wallaroo follows these steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and set up the workspace.
* [Upload The Model](#upload-the-model): Upload the model and autoconvert for use in the Wallaroo engine.
* [Upload the Processing Modules](#upload-the-processing-modules): Upload the processing modules.
* [Create and Deploy the Pipeline](#create-and-deploy-the-pipeline): Create the pipeline with the model and processing modules as steps, then deploy it.
* [Test the Pipeline](#test-the-pipeline): Verify that the pipeline works with the sample data.

### Connect to Wallaroo

First we import the required libraries to connect to the Wallaroo instance, then connect to the Wallaroo instance.

In [1]:
import json
import pickle
import pandas as pd
import numpy as np

import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
os.environ["ARROW_ENABLED"]="True"

import simdb # module for the purpose of this demo to simulate pulling data from a database

from wallaroo.ModelConversion import ConvertXGBoostArgs, ModelConversionSource, ModelConversionInputType
import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import preprocess

import postprocess

In [2]:
# Login through local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR PREFIX"

wallarooPrefix = "doc-test"
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

Please log into the following URL in a web browser:

	https://doc-test.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=FSJN-OZBJ

Login successful!


In [3]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [4]:
workspace_name = 'housepricing'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

The workspace `housepricing` will either be created or, if already existing, used and set to the current workspace.

In [5]:
new_workspace = get_workspace(workspace_name)
new_workspace

{'name': 'housepricing', 'id': 9, 'archived': False, 'created_by': 'bed53c89-7e30-496b-a5d2-4d9289cb5eb4', 'created_at': '2023-04-26T18:01:52.439309+00:00', 'models': [], 'pipelines': []}

In [6]:
_ = wl.set_current_workspace(new_workspace)

### Upload The Model

With the connection set and workspace prepared, upload the model created in `02_automated_training_process.ipynb` into the current workspace.

In [7]:
hpmodel = wl.upload_model(model_name, model_file).configure()

### Upload the Processing Modules

Upload the preprocess.py and postprocess.py modules as models to be added to the pipeline.

In [9]:
# load the preprocess module
module_pre = wl.upload_model("preprocess", "./preprocess.py").configure('python')

# load the postprocess module
module_post = wl.upload_model("postprocess", "./postprocess.py").configure('python')

### Create and Deploy the Pipeline

Create the pipeline with the preprocess module, housing model, and postprocess module as pipeline steps, then deploy the newpipeline.

In [10]:
pipeline = (wl.build_pipeline(pipeline_name)
              .add_model_step(module_pre)
              .add_model_step(hpmodel)
              .add_model_step(module_post)
              .deploy()
           )
pipeline

name,housing-pipe
created,2023-04-26 18:01:55.793655+00:00
last_updated,2023-04-26 18:03:41.202098+00:00
deployed,True
tags,
versions,"af4b912a-16d5-4177-a040-7576c0ff16ca, 488b14f4-2231-4717-a0d7-2d2250d4411a, 34f3213d-6f4b-4df3-8cd5-1f665c24f99a"
steps,preprocess


### Test the Pipeline

We will use a single query from the simulated `housing_price` table and infer.  Since the model expects a numpy array, we'll convert the data as that then submit it as a DataFrame, then convert the result back into a regular price we can use.

When successful, we will undeploy the pipeline to restore the resources back to the Kubernetes environment.

In [14]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} limit 1"
print(query)

# read in the data
singleton = pd.read_sql_query(query, conn)
conn.close()

# preprocessed = pd.DataFrame.from_records({'tensor': preprocess.create_features(singleton)})

# display(preprocessed)

select * from house_listings limit 1


,id,date,list_price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sale_price
0,7129300520,2022-09-12,221900.0,3,1.0,1180,5650,1.0,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900.0


In [17]:
result = pipeline.infer(preprocessed)

# turn data into a file

# with open('convert.json', 'w') as convert_file:
#      convert_file.write(json.dumps({'query': singleton.to_json()}))

# result = pipeline.infer_from_file("./convert.json", data_format="custom-json")

# # result = pipeline.infer({'query': singleton.to_json()})
display(result)

InferenceError: Inference failed: 
	InternalError: Error running inference <class 'AttributeError'>, 'list' object has no attribute 'to_numpy'

In [ ]:
# prediction = postprocess.postprocess(result.loc[0,"out.variable"][0])
# display(prediction)

When finished, we undeploy the pipeline to return the resources back to the environment.

In [18]:
pipeline.undeploy()

Please log into the following URL in a web browser:

	https://doc-test.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=MBDM-BQRB

Login successful!


RuntimeError: Undeployment failed. See status for details.
Status: {'status': 'Error', 'details': [], 'engines': [{'ip': '10.100.3.49', 'name': 'engine-6bb4486c57-sqj96', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': None, 'model_statuses': None}], 'engine_lbs': [{'ip': '10.100.3.48', 'name': 'engine-lb-ddd995646-sj4kx', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}

With this stage complete, we can proceed to Stage 4: Regular Batch Inference.